In [1]:
import socks
import socket
import requests
from bs4 import BeautifulSoup
import time
import re
import numpy as np
import pandas as pd

socks.set_default_proxy(socks.SOCKS5, "localhost", 9050) # one source said port 9150, Georgia's code used 9050
socket.socket = socks.socksocket

# These lines are copied from the Linkedin post.
def getaddrinfo(*args):
    return [(socket.AF_INET, socket.SOCK_STREAM, 6, '', (args[0], args[1]))]
socket.getaddrinfo = getaddrinfo

# ToRReZ market URL
url = "http://yxuy5oau7nugw4kpb4lclrqdbixp3wvc4iuiad23ebyp2q3gx7rtrgqd.onion/items/category/drugs-and-chemicals?order=none&vendors=all&shipping_to=any&shipping_from=any&product_type=any&payments_method=any&page=1"


# Manually selected from the browser header after solving the captcha
cookies_copied = 'Cookie: hr4ujvby8ds459og4kzcpbzwjdj_session=eyJpdiI6ImZZMmVSYVo3Z0s4WWxuMWlpQ1pmXC9BPT0iLCJ2YWx1ZSI6IlNpSWwydWl5Tks0M1NJTDREcE03Y3VxbEFYaEVYU1daelVoeHZhR0QyKzV3ZlZyKzFMcmlIXC9KR1wvaGFIY1pJTVVyWCtKMVVWUkVmXC9LOXVhS3c2MktOQlJ4TzVkK2VweVVjbGxiWVUwQ2EyK3J2UjFyVG9saWV5RjlBREdFc09vIiwibWFjIjoiNWVjYmEwZTk0MGQxMWU2YTk4OGYyZWU2OTRlMjIxYzhhZWI1NDEwZmQwNjg1NGIzMGFhY2ViZTE1OWU0Y2E1OCJ9; XSRF-TOKEN=eyJpdiI6IktTVnB6dExwbDZOT3FWRzVsb0tCSmc9PSIsInZhbHVlIjoiMERsYnJabHY5Z2UrdVpicGR1eHlNZGpKbUhSVmZ0cEZnckI0MUtJdkJleVpWNDhUSWp4T1wvM1cwWGlqdVdORVo4OFwvR0thRTBcL2JuaDEwNFQzV21VSnNXRWxLXC9zNDRGQzU1aGVWMDhqVlJLTGs2alNSXC9POWxHY0lGckVPUFFPMCIsIm1hYyI6ImVhZGVjNTBkNzFhZmE1NTdjOTExMWM2ZmUxNmFjNjliOGU5ZWM4MTI4MGJlNDY3ZGFiNjFiZmU3NjMyNDA1YjgifQ%3D%3D; ray_id=945db2f96a6f105f6531684e18253aa01528a1b548a9b6b9f36c21125d02afb0'


# Takes the relevant info of the copied cookies and puts it in the correct format automatically.
def convert_pasted_cookies_to_usable_format(pasted_cookies):
    all_cookies_str = pasted_cookies.split(':')[1] # to ignore the prefix "Cookie: ")
    individual_cookies = all_cookies_str.split(';')
    cookies_set = {}
    for cookie in individual_cookies:
        cookies_set.update({cookie.strip().split('=')[0]: cookie.strip().split('=')[1]})
    return cookies_set


s = requests.Session()
s.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:78.0) Gecko/20100101 Firefox/78.0'
})

proxies = {
    'http': 'socks5h://localhost:9050',
    'https': 'socks5h://localhost:9050'
}

set_of_cookies = convert_pasted_cookies_to_usable_format(cookies_copied)

# Get the page category drugs
web_page = s.get(url, cookies=set_of_cookies, proxies=proxies)
soup = BeautifulSoup(web_page.content, 'html.parser')



# create empty lists
vendor = []    
vendor_offers = []
all_titles = []
all_quantities = []
all_prices = []
category = []
category_level_1 = []
category_level_2 = []
category_level_3 = []
ship_from = []
ship_to =[]



# crawl and scrape through pages
pages = []
for a in soup.find_all('a',attrs={"class" : "page-link"}):
    pages.append(a.text)
last_page = pages[-2]

for page in range(1,3): #last_page+1

    url = 'http://yxuy5oau7nugw4kpb4lclrqdbixp3wvc4iuiad23ebyp2q3gx7rtrgqd.onion/items/category/drugs-and-chemicals?order=none&vendors=all&shipping_to=any&shipping_from=any&product_type=any&payments_method=any&page=' + str(page)

    web_page = s.get(url, cookies=set_of_cookies, proxies=proxies)
    soup = BeautifulSoup(web_page.content, 'html.parser')


    for a in soup.find_all('a', href=True, class_='title'):
        all_titles.append(a.text.strip())
        if "gram" in a.text.strip().lower():
            #print(a.text.strip())
            if "." in a.text.strip().lower().split('gram')[0]:
                quantity = (re.findall("\d+\.\d+", a.text.strip().lower().split('gram')[0]))[0]
            else:
                quantity = (re.findall("\d+", a.text.strip().lower().split('gram')[0]))[0]
        else:
            #print("No quantity found in: " + a.text.strip())
            quantity = 0

        all_quantities.append(quantity)

    for price in soup.find_all('span', class_='Price'):
        all_prices.append(re.findall("\d+\.\d+", price.text.strip())[0])

    for a in soup.find_all('div',attrs={"class" : "w-100 mt-3 small"}):
        if len(a.find_all('a'))==4:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(a.find_all('a')[2].text.strip())
            category_level_3.append(a.find_all('a')[3].text.strip())
        if len(a.find_all('a'))==3:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(a.find_all('a')[2].text.strip())
            category_level_3.append(np.nan)
        if len(a.find_all('a'))==2:
            category_level_1.append(a.find_all('a')[1].text.strip())
            category_level_2.append(np.nan)
            category_level_3.append(np.nan)
        last_item = a.find_all('a')[-1]
        category.append(last_item.text.strip())

        
    # deze nog niet klaar    
    for ship in soup.find_all('span', class_='shippingFrom'):
        ship_from.append(ship.text.strip())


    for ship in soup.find_all('span', class_='shippingTo'):
        ship_to.append(ship.text.strip())  

    for a in soup.find_all('a', class_='font-weight-bold'):
        vendor.append(a.text.strip())


            
            
results = (list(zip(vendor, category_level_1, category_level_2, category_level_3, category, all_titles, all_quantities, all_prices, ship_from, ship_to)))



df = pd.DataFrame(results,columns=['vendor', 'category_level_1', 'category_level_2', 'category_level_3', 'highest_category', 'product','quantity','price', 'shipping_from', 'shipping_to'])    


In [2]:
df

,vendor,category_level_1,category_level_2,category_level_3,highest_category,product,quantity,price,shipping_from,shipping_to
0,PremierLeague,Drugs and Chemicals,Opiates,Oxycodone,Oxycodone,7x Pills OXYCODONE M30 (30MG) HQ Pressd,0,68.37,United Kingdom,United Kingdom
1,SmartStore,Drugs and Chemicals,Benzos,NaN,Benzos,Xanax (Alprazolam) Original 2 mg x 100 tablets,0,108.43,United Kingdom,Europe
2,Picasso,Drugs and Chemicals,Stimulants,Speed,Speed,5g Amphetaminen Speed Paste AAA+++,0,30.12,Germany,Germany
3,BritishStandard,Drugs and Chemicals,Benzos,Pills,Pills,9000x 3mg R039 xanax Actavis Pharmaceuticals,0,385.85,United Kingdom,Worldwide
4,probitexchange,Drugs and Chemicals,Benzos,Pills,Pills,100 x Clonazepam Klonopin 2mg Generic,0,499.00,United States,United States
5,toyplay007,Drugs and Chemicals,Cannabis & Hash,NaN,Cannabis & Hash,Buy CBD Powder 99.9% Purity Online (500g ),0,000.00,China,Worldwide
6,QUICKDRUGSUK,Drugs and Chemicals,NaN,NaN,Drugs and Chemicals,7G PERUVIAN COCAINE HIGH PURITY,0,460.43,United Kingdom,Worldwide
7,DutchGlory,Drugs and Chemicals,Stimulants,Cocaine,Cocaine,(2G) - Colombian Cocaine *Pure/Uncut*,0,132.53,Netherlands,Worldwide
8,CanadianSmoker12,Drugs and Chemicals,Cannabis & Hash,Buds & Flowers,Buds & Flowers,3 pounds A+++ Amnesia Haze,0,500.00,United States,North America
9,TheBodyShop,Drugs and Chemicals,Prescriptions Drugs,NaN,Prescriptions Drugs,+EUROPE RETAIL+ CARISOPRODOL Soma 500mg 100 tabs,0,105.00,Netherlands,Worldwide
